<a href="https://colab.research.google.com/github/Shak97/Getting-and-Cleaning-Data-Course-Project/blob/master/Copy_of_Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Activation, Dropout, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
import itertools
import os

In [0]:
from keras import backend as K

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os

In [0]:
import cv2

def data_gen(img_folder, mask_folder, batch_size):
  c = 0
  n = os.listdir(img_folder) #List of training images
  #random.shuffle(n)
  
  while (True):
    img = np.zeros((batch_size, 320, 320, 1)).astype('float')
    mask = np.zeros((batch_size, 320, 320, 1)).astype('float')

    for i in range(c, c+batch_size): #initially from 0 to 16, c = 0. 

      train_img = cv2.imread(img_folder+'/'+n[i])/255.
      train_img =  cv2.resize(train_img, (320, 320))# Read an image from folder and resize
      
      img[i-c] = train_img #add to array - img[0], img[1], and so on.
                                                   

      train_mask = cv2.imread(mask_folder+'/'+n[i], cv2.IMREAD_GRAYSCALE)/255.
      train_mask = cv2.resize(train_mask, (320, 320))
      train_mask = train_mask.reshape(320, 320, 1) # Add extra dimension for parity with train_img size [512 * 512 * 3]

      mask[i-c] = train_mask

    c+=batch_size
    if(c+batch_size>=len(os.listdir(img_folder))):
      c=0
      random.shuffle(n)
                  # print "randomizing again"
    yield img, mask

In [0]:
train_frame_path = '/content/drive/My Drive/h_segmentation/train_image/images/'
train_mask_path = '/content/drive/My Drive/h_segmentation/train_mask/labels/'


# Train the model
train_gen = data_gen(train_frame_path,train_mask_path, batch_size = 2251)

In [0]:
train_gen

<generator object data_gen at 0x7f84d3c54518>

In [0]:
train_image = ImageDataGenerator(rescale=1./255).flow_from_directory('/content/drive/My Drive/h_segmentation/train_image/',batch_size = 2251, target_size = (320,320), seed = 3)

Found 2251 images belonging to 1 classes.


In [0]:
train_image

In [0]:
train_mask = ImageDataGenerator(rescale=1./255).flow_from_directory('/content/drive/My Drive/h_segmentation/train_mask/',batch_size=2251, target_size = (320,320), seed = 3)

Found 2251 images belonging to 1 classes.


In [0]:
train_generator = zip(train_image, train_mask)

In [0]:
def unet(weights = None, input_size = (320,320,1)):
  
  inp = keras.engine.input_layer.Input(input_size)
  c1 = Conv2D(64, (3,3), activation='relu', padding='same')(inp)
  c1 = BatchNormalization()(c1)
  c1 = Conv2D(64, (3,3), activation = 'relu', padding = 'same')(c1)
  c1 = BatchNormalization()(c1)
  p1 = MaxPooling2D(pool_size=(2,2))(c1)
  #p1 = Dropout(0.1)(p1)
  
  c2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(p1)
  c2 = BatchNormalization()(c2)
  c2 = Conv2D(128, (3,3), activation = 'relu', padding = 'same')(c2)
  c2 = BatchNormalization()(c2)
  p2 = MaxPooling2D(pool_size=(2,2))(c2)
  #p2 = Dropout(0.1)(p2)
  
  c3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(p2)
  c3 = BatchNormalization()(c3)
  c3 = Conv2D(256, (3,3), activation = 'relu', padding = 'same')(c3)
  c3 = BatchNormalization()(c3)
  p3 = MaxPooling2D(pool_size=(2,2))(c3)
  #p3 = Dropout(0.1)(p3)
  
  c4 = Conv2D(512, (3,3), activation = 'relu', padding='same')(p3)
  c4 = BatchNormalization()(c4)
  c4 = Conv2D(512, (3,3), activation = 'relu', padding = 'same')(c4)
  c4 = BatchNormalization()(c4)
  p4 = MaxPooling2D(pool_size=(2,2))(c4)
  #p4 = Dropout(0.1)(p4)
  
  c5 = Conv2D(1024, (3,3), activation = 'relu', padding='same')(p4)
  c5 = BatchNormalization()(c5)
  c5 = Conv2D(1024, (3,3), activation = 'relu', padding = 'same')(c5)
  c5 = BatchNormalization()(c5)
  #p5 = MaxPooling2D(pool_size=(2,2))(c5)
  
  u6 = Conv2DTranspose(512, (3,3), activation = 'relu', strides = (2,2), padding = 'same')(c5)
  concat6 = concatenate([u6,c4])
  c6 = Conv2D(512, (3,3), activation = 'relu', padding = 'same')(concat6)
  c6 = BatchNormalization()(c6)
  c6 = Conv2D(512, (3,3), activation = 'relu', padding = 'same')(c6)
  c6 = BatchNormalization()(c6)
  
  u7 = Conv2DTranspose(256, (3,3), activation = 'relu', strides = (2,2), padding = 'same')(c6)
  concat7 = concatenate([u7,c3])
  c7 = Conv2D(256, (3,3), activation = 'relu', padding = 'same')(concat7)
  c7 = BatchNormalization()(c7)
  c7 = Conv2D(256, (3,3), activation = 'relu', padding = 'same')(c7)
  c7 = BatchNormalization()(c7)
  
  u8 = Conv2DTranspose(128, (3,3), activation = 'relu', strides = (2,2), padding = 'same')(c7)
  concat8 = concatenate([u8,c2])
  c8 = Conv2D(128, (3,3), activation = 'relu', padding = 'same')(concat8)
  c8 = BatchNormalization()(c8)
  c8 = Conv2D(128, (3,3), activation = 'relu', padding = 'same')(c8)
  c8 = BatchNormalization()(c8)
  
  u9 = Conv2DTranspose(64, (3,3), activation = 'relu', strides = (2,2), padding = 'same')(c8)
  concat9 = concatenate([u9,c1])
  c9 = Conv2D(64, (3,3), activation = 'relu', padding = 'same')(concat9)
  c9 = BatchNormalization()(c9)
  c9 = Conv2D(64, (3,3), activation = 'relu', padding = 'same')(c9)
  c9 = BatchNormalization()(c9)
  
  c10 = Conv2D(1, (1,1), activation = 'sigmoid')(c9)
  
  model = Model(input=inp, output=c10)   #input passed to this function must come from keras.layers.Input(a tensor), otherwise it will through error
  return model
  
  
  
  
  
  

In [0]:
  def iou_mat(y_true, y_pred, smooth = 1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true,-1) + K.sum(y_pred,-1) - intersection
    iou = (intersection + smooth) / ( union + smooth)
    return iou
  
  def iou_coef_loss(y_true, y_pred):
    return 1-iou_mat(y_true, y_pred)

In [0]:
model = unet()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [0]:
model.compile(optimizer=Adam(), loss=iou_coef_loss, metrics=[iou_mat])

In [0]:
model.fit_generator(train_generator, steps_per_epoch = 2251, epochs = 15, verbose=2)

Epoch 1/15


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:709: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)


In [0]:
import PIL
from PIL import Image
import nibabel as nib
from matplotlib import image

In [0]:
im = Image.open('drive/My Drive/h_segmentation/train_image/images/la_003image1.png')

In [0]:
a = im.get_fdata()

AttributeError: ignored

In [0]:
np.array(im.getchannel(0)).shape

(320, 320)

In [0]:
data = np.asarray(im)

In [0]:
print(data.reshape(320,320,1))

[[[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 ...

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  ...
  [0]
  [0]
  [0]]]
